# Baselines for CS224U Project

## Setup

### Imports

In [42]:
from sklearn.model_selection import train_test_split, GridSearchCV
import csv
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [3]:
import utils
import sst
import scipy.stats
from sgd_classifier import BasicSGDClassifier

In [6]:
from sklearn.linear_model import LogisticRegression
import os

In [55]:
import tensorflow as tf
from tf_rnn_classifier import TfRNNClassifier

/home/declan/anaconda3/envs/nlu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
vsmdata_home = 'vsmdata'

glove_home = os.path.join(vsmdata_home, 'glove.6B')

### Dataset

In [8]:
def read_array_from_csv(inputcsv):
    out = []
    with open(inputcsv, encoding='utf-8') as f:
        reader = csv.reader(f)
        for q in reader:
            out.append(q)
    return out

In [9]:
new_train = read_array_from_csv('data/train_data.csv')

In [10]:
anon_new_train = read_array_from_csv('data/anon_train_data.csv')

In [11]:
anon_new_test = read_array_from_csv('data/anon_test_data.csv')

### SST Machinery

In [12]:
def hansard_reader(
        src_filename,
        class_func=None):
    """Overview

    Parameters
    ----------
    src_filename : str
        Full path to the file to be read.
    class_func : None, or function mapping labels to labels or None
        If this is None, then the original 5-way labels are returned.
        Other options: `binary_class_func` and `ternary_class_func`
        (or you could write your own).


    Yields
    ------
    (tree, label)
        nltk.Tree, str in {'0','1','2','3','4'}

    """
    if class_func is None:
        class_func = lambda x: x
    with open(src_filename, encoding='utf-8') as f:
        reader = csv.reader(f)
        for q in reader:
            yield (q[0], class_func(q[1]))

We need a reader for each dataset, both for train and for test.

First, the standard data:

In [13]:
def train_reader(**kwargs):
    """Convenience function for reading the train file, full-trees only."""
    src = 'data/train_data.csv'
    return hansard_reader(src,**kwargs)

In [14]:
def test_reader(**kwargs):
    """Convenience function for reading the train file, full-trees only."""
    src = 'data/test_data.csv'
    return hansard_reader(src,**kwargs)

Next, the anonymised data:

In [15]:
def anon_train_reader(**kwargs):
    src = 'data/anon_train_data.csv'
    return hansard_reader(src,**kwargs)

In [16]:
def anon_test_reader(**kwargs):
    src = 'data/anon_test_data.csv'
    return hansard_reader(src,**kwargs)

The test readers won't be used until the *very* end.

### Class Functions

In [17]:
def cas_to_gov(label):
    if label == 'cas':
        return 'gov'
    else:
        return label

### Bag-of-words Feature Functions

A unigrams feature function

In [19]:
def unigrams_phi(question):
    """The basis for a unigrams feature function.
    Parameters
    ----------
    question : string
        The question to represent.
    
    Returns
    -------    
    defaultdict
        A map from strings to their counts in the question. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    """
    unigrams = {}
    for word in question.split() :
        unigrams[word] = unigrams.get(word, 0) + 1
    return unigrams

A bigrams feature function

In [9]:
def bigrams_phi(question):
    """The basis for a unigrams feature function.
    
    Parameters
    ----------
    tree : nltk.tree
        The tree to represent.
    
    Returns
    -------    
    defaultdict
        A map from strings to their counts in `tree`. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    
    """
    bigrams = {}
    qarray = question.split()
    for i in range(0, len(qarray)-1) :
        big = qarray[i] + '_' + qarray[i+1]
        bigrams[big] = bigrams.get(big, 0) + 1
    return bigrams

A basic bag-of-words unigrams and bigrams feature function

In [11]:
def uni_bigrams_phi(question):
    grams = unigrams_phi(question)
    grams.update(bigrams_phi(question))
    return grams

We found that *friend* seems to be a good indicator. What happens if we only give the classifier that feature? Or unigrams without it?

In [19]:
def only_friend_phi(question):
    if 'friend' in question.lower().split():
        return {'friend':1}
    else:
        return {'friend':0}

In [20]:
def no_friends_phi(question):
    unigrams = {}
    for word in question.split() :
        if word.lower() is not 'friend':
            unigrams[word.lower()] = unigrams.get(word.lower(), 0) + 1
    return unigrams

### SGD Classifier Baseline
Stochastic Gradient Descent

### Logistic Regression without using sst.experiment

#### Setting up parameters

First, choose the feature function to use.

In [20]:
phi = unigrams_phi

Next, choose the reader used for testing. (None gives a random split.)

In [21]:
assess_reader = None

If we're doing a split, what size should we train on?

In [22]:
train_size = 0.7

Next, choose a function for the classes. (We probably want cas_to_gov.)

In [23]:
class_func = cas_to_gov

Do we want to vectorise?

In [24]:
vectorize = True

#### Building the Classifier

Which classifier are we to use?

In [61]:
classifier = LogisticRegression(fit_intercept=True)

Make it into a training function.

In [62]:
def train_func(X, y):   
    mod = classifier
    mod.fit(X, y)
    return mod

Build the dataset.

In [25]:
train = sst.build_dataset(train_reader, phi, class_func, vectorize=vectorize)

#### Setting up the experiment

First, get the data into standardised variables.

In [26]:
X_train = train['X']
y_train = train['y']
X_assess = None
y_assess = None

If we're not using an assess_reader, do a split on the training data. Otherwise, read in the assessment dataset.

In [27]:
if assess_reader == None:
     X_train, X_assess, y_train, y_assess = train_test_split(
            X_train, y_train, train_size=train_size, test_size=None)
else:
    # Assessment dataset using the training vectorizer:
    assess = sst.build_dataset(
        assess_reader,
        phi,
        class_func,
        vectorizer=train['vectorizer'],
        vectorize=vectorize)
    X_assess, y_assess = assess['X'], assess['y']

#### Experiment
Train the model.

In [ ]:
mod = train_func(X_train, y_train)

Test the model.

In [ ]:
predictions = mod.predict(X_assess)

Print a report.

In [46]:
print('Accuracy: %0.03f' % sst.accuracy_score(y_assess, predictions))
print(classification_report(y_assess, predictions, digits=3))

Accuracy: 0.768
             precision    recall  f1-score   support

        gov      0.677     0.603     0.638      1020
        opp      0.807     0.853     0.829      1987

avg / total      0.763     0.768     0.764      3007



## Logistic Regression with GloVe

This should be fairly straightforward. We do Logistic Regression again, but we use GloVe embeddings, rather than the bag of words embeddings from above.

### GloVe Setup

First, we need the GloVe lookup function.

In [ ]:
glove_lookup = utils.glove2dict(
    os.path.join(glove_home, 'glove.6B.50d.txt'))

Now build a feature function based on the GloVe embeddings. The important parameter here is how we combine vectors for different words.

In [ ]:
def vsm_words_phi(sentence, lookup, np_func=np.sum):
    """Represent `sentence` as a combination of the vector of its words.
    
    Parameters
    ----------
    tree : A string   
    lookup : dict
        From words to vectors.
    np_func : function (default: np.sum)
        A numpy matrix operation that can be applied columnwise, 
        like `np.mean`, `np.sum`, or `np.prod`. The requirement is that 
        the function take `axis=0` as one of its arguments (to ensure
        columnwise combination) and that it return a vector of a 
        fixed length, no matter what the size of the tree is.
    
    Returns
    -------
    np.array, dimension `X.shape[1]`
            
    """      
    allvecs = np.array([lookup[w] for w in sentence.split() if w in lookup])    
    if len(allvecs) == 0:
        dim = len(next(iter(lookup.values())))
        feats = np.zeros(dim)
    else:       
        feats = np_func(allvecs, axis=0)      
    return feats

In [ ]:
def glove_words_phi(sentence, np_func=np.sum):
    return vsm_words_phi(sentence, glove_lookup, np_func=np_func)

### Experiment

For this, we'll do some grid searching. In order to show how to make this work in general, here we'll step through the process.

#### Setting Parameters

We need a base model to do a grid search on.

In [ ]:
basemod = LogisticRegression()

We need to specify a parameter grid to search over.

In [ ]:
param_grid = {'fit_intercept': [True, False], 
                  'C': [0.4, 0.6, 0.8, 1.0, 2.0, 3.0],
                  'penalty': ['l1','l2']}

How many fold cross-validation? (Default is None.)

In [ ]:
cv = 5

What score metric should be used? (Some function is required here, unless the basemod provides its own.). Options include 'f1_macro', 'f1_micro' and 'accuracy'.

In [ ]:
scoring = 'f1_macro'

#### Build the GridSearch

In [ ]:
crossvalidator = GridSearchCV(basemod, param_grid, cv=cv, scoring=scoring)
crossvalidator.fit(X_train, y_train)

#### Get some reports

In [ ]:
print("Best params", crossvalidator.best_params_)
print("Best score: %0.03f" % crossvalidator.best_score_)

#### Final Model

In [ ]:
final_mod = crossvalidator.best_estimator_

Test the model.

In [ ]:
predictions = final_mod.predict(X_assess)

Print a report.

In [46]:
print('Accuracy: %0.03f' % sst.accuracy_score(y_assess, predictions))
print(classification_report(y_assess, predictions, digits=3))

Accuracy: 0.768
             precision    recall  f1-score   support

        gov      0.677     0.603     0.638      1020
        opp      0.807     0.853     0.829      1987

avg / total      0.763     0.768     0.764      3007



## Recurrent Neural Network

This should use the tensorflow RNN set-up.

### Build the Input Vectors

In [46]:
train_data = [(question.split(), label) for question, label in train_reader(class_func=cas_to_gov)]
X, y = zip(*train_data)
X_rnn_train = list(X)
y_rnn_train = list(y)

Since we don't have a devoted dev set, make a 70-30 split in the training data

In [47]:
X_rnn_train, X_rnn_assess, y_rnn_train, y_rnn_assess = train_test_split(X_rnn_train, y_rnn_train, train_size=0.7, test_size=None)

### Analyse the Inputs

The tensorflow implementation requires that we specify a maximum length up front.

In [48]:
utils.sequence_length_report(X_rnn_train, potential_max_length=150)

Max sequence length: 1,123
Min sequence length: 0
Mean sequence length: 71.78
Median sequence length: 69.00
Sequences longer than 150: 225 of 7,115


Based on this, we might take this maximum length to be 150.

#### Get the vocab

In [49]:
hansard_full_train_vocab = sst.get_vocab(X_rnn_train)

In [52]:
print("hansard_full_train_vocab has {:,} items".format(len(hansard_full_train_vocab)))

hansard_full_train_vocab has 35,337 items


In [53]:
hansard_train_vocab = sst.get_vocab(X_rnn_train, n_words=5000)

### Experiments

#### Random Embeddings

In [83]:
tf_rnn = TfRNNClassifier(
    hansard_train_vocab,
    embed_dim=50,
    hidden_dim=50,
    max_length=150,
    hidden_activation=tf.nn.tanh,
    cell_class=tf.nn.rnn_cell.LSTMCell,
    train_embedding=True,
    max_iter=10,
    eta=0.05) 

In [ ]:
_ = tf_rnn.fit(X_rnn_train, y_rnn_train)

Iteration 5: loss: 4.5841029286384586

In [ ]:
tf_rnn_dev_predictions = tf_rnn.predict(X_rnn_assess)

In [ ]:
print(classification_report(y_rnn_dev, rnn_dev_predictions))

## Convolutional Neural Network

## Long/Short Term Memory Neural Network